In [2]:
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
raw_data = pd.read_json("farmers-protest-tweets-2021-03-5.json", lines = True)
tweets = data_preprocessing(raw_data)
tw

In [ ]:
def main():
    print("Loading data. This may take a few minutes...")
    raw_data = pd.read_json("farmers-protest-tweets-2021-03-5.json", lines = True)
    print("Data loaded. Preprocessing data...")
    tweets = data_preprocessing(raw_data)
    print("Data processed.")
    query = input("Please enter query ID (1, 2, 3, 4)")
    if query == "1":
        print("Top 10 most retweeted:")
    elif query == "2":
        print("Top 10 users")
    

In [ ]:
def data_preprocessing(data):
    users = json_normalize(data["user"])
    users.rename(columns={"id":"userId", "url":"profileUrl"}, inplace = True)
    users = pd.DataFrame(users)
    users.drop_duplicates(subset = ["userId"], inplace = True)
    user_id = []
    for user in data["user"]:
        uid = user["id"]
        user_id.append(uid)
    data["userId"] = user_id
    return data